### IS_602 HW12
Daina Bouquin

1. Use your solution from Homework 11 and complete the Monte Carlo step in parallel. 

2. Compare the timing for your solution in homework 11 and this parallel solution.

##### Re-running the initial analysis from HW11

In [2]:
import Tkinter
import tkFileDialog
import pandas as pd
import numpy as np
from pandas import Series
import os
import timeit
import ipyparallel # parallel was depreciated so 'pip install ipyparallel'

In [3]:
root = Tkinter.Tk()
root.withdraw()
file_p = tkFileDialog.askopenfilename(parent=root) # select the apple data file
AD = pd.read_csv(file_p)
AD.columns = ['date','price','p_change']

In [4]:
# Analysis from HW11
p = Series(AD['p_change'][1:],dtype=float) # not using first row (headers)
mu, sigma = np.mean(p), np.std(p)
mu, sigma

(0.0009573552071713143, 0.016487613979893456)

In [5]:
# Monte Carlo from HW11
f_price = []
for i in range(0,10000):
    l_price = AD['price'][251] # the last row (we aren't looking at the headers)
    diff_20 = np.random.normal(mu, sigma, 20)
    next_20 = [] # nesting to keep it simple
    for e in diff_20:
        l_price = l_price + l_price * e #notation shared by David Stern
        next_20.append(l_price)
    f_price.append(next_20[19])

print np.percentile(f_price,1) 

346.507305995


##### Timing the first approach to Monte Carlo from HW11

In [6]:
%timeit diff_20
f_price = []
for i in range(0,10000):
    l_price = AD['price'][251] # the last row (we aren't looking at the headers)
    diff_20 = np.random.normal(mu, sigma, 20)
    next_20 = [] # nesting to keep it simple
    for e in diff_20:
        l_price = l_price + l_price * e #notation shared by David Stern
        next_20.append(l_price)
    f_price.append(next_20[19])

print np.percentile(f_price,1) 

10000000 loops, best of 3: 27.1 ns per loop
348.135750146


##### Parallel Processing approach to the Monte Carlo

In [11]:
# In terminal start 4 clusters: ipcluster start -n 4
# In terminal: ipython profile create --parallel --profile=myprofile
from ipyparallel import Client

In [12]:
c = Client()

In [13]:
clients = ipyparallel.Client()
dview = clients.direct_view()

In [15]:
%%px 
import Tkinter
import tkFileDialog
import pandas as pd
import numpy as np
from pandas import Series
import os
import timeit
import ipyparallel

root = Tkinter.Tk()
root.withdraw()
file_p = tkFileDialog.askopenfilename(parent=root) # select the apple data file
AD = pd.read_csv(file_p)
AD.columns = ['date','price','p_change']

# Analysis from HW11
p = Series(AD['p_change'][1:],dtype=float) # not using first row (headers)
mu, sigma = np.mean(p), np.std(p)
mu, sigma

# the parallel part
clients = ipyparallel.Client()
dview = clients.direct_view()

def rand20(mean,std):
    import random
    randomlist = []
    for i in range(0,20):
         randomlist.append(random.gauss(mean,std))
    return randomlist

In [17]:
c.ids # checking to make sure there are 4 clusters

[0, 1, 2, 3]

In [16]:
%%px 
f_price = []
diff_20 = []

%timeit diff_20
for i in range(0,10000):
    l_price = AD['price'][251] # the last row (we aren't looking at the headers)
    diff_20 = np.random.normal(mu, sigma, 20)
    next_20 = [] # nesting to keep it simple
    for e in diff_20:
        l_price = l_price + l_price * e #notation shared by David Stern
        next_20.append(l_price)
    f_price.append(next_20[19])

print np.percentile(f_price,1) 

[stdout:0] 
10000000 loops, best of 3: 45.9 ns per loop
346.32858915
[stdout:1] 
10000000 loops, best of 3: 46.3 ns per loop
347.52744757
[stdout:2] 
10000000 loops, best of 3: 46.3 ns per loop
347.211661754
[stdout:3] 
10000000 loops, best of 3: 46.3 ns per loop
346.792277847
